In [1]:
import pandas as pd
import json
import time
from google_currency import convert 
import warnings
warnings.filterwarnings('ignore')


In [2]:
df=pd.read_csv('/Users/mikel/Documents/Projects/chameleon-pricing/country_currency.csv')
df.head()

,Country,last_value,Currency,Code
0,aruba,1.350690,Aruban Florin,AWG
1,afghanistan,17.914483,Afghani,AFN
2,angola,146.167168,Kwanza,AOA
3,albania,41.342391,Lek,ALL
4,united arab emirates,2.258503,UAE Dirham,AED


# PPP to dollar

https://pypi.org/project/google-currency/

!pip install google-currency

In [3]:
def ppp_to_dollar(Code, last_value):
        conversion=convert(Code, 'usd', last_value)
        result = json.loads(conversion)
        return result['amount']

In [4]:
ppp_to_dollar('EUR', 8229.53)

'9825.44'

In [5]:
%%time
apply = df.apply(
         lambda x: ppp_to_dollar(x['Code'], x['last_value']),
         axis=1)
df['ppp_todollar']  = apply
print(df)

Invalid currency codes passed in parameters, original exception message is -> 'BOV'
Invalid currency codes passed in parameters, original exception message is -> 'CHE'
Unable to convert currency, failed to fetch results from Google
Invalid currency codes passed in parameters, original exception message is -> 'CHW'
Invalid currency codes passed in parameters, original exception message is -> 'COU'
Invalid currency codes passed in parameters, original exception message is -> 'CUC'
Invalid currency codes passed in parameters, original exception message is -> 'ERN'
Invalid currency codes passed in parameters, original exception message is -> 'GHS'
Invalid currency codes passed in parameters, original exception message is -> 'MXV'
Invalid currency codes passed in parameters, original exception message is -> 'MNT'
Invalid currency codes passed in parameters, original exception message is -> 'MZN'
Invalid currency codes passed in parameters, original exception message is -> 'RON'
Invalid curr

                  Country  last_value         Currency Code ppp_todollar
0                   aruba    1.350690    Aruban Florin  AWG         0.75
1             afghanistan   17.914483          Afghani  AFN         0.23
2                  angola  146.167168           Kwanza  AOA         0.22
3                 albania   41.342391              Lek  ALL         0.40
4    united arab emirates    2.258503       UAE Dirham  AED         0.61
..                    ...         ...              ...  ...          ...
183               vanuatu  108.104856             Vatu  VUV            0
184                 samoa    1.662674             Tala  WST            0
185          south africa    6.652566             Rand  ZAR         0.47
186                zambia    4.641657   Zambian Kwacha  ZMW         0.26
187              zimbabwe    0.494348  Zimbabwe Dollar  ZWL            0

[188 rows x 5 columns]
CPU times: user 3.17 s, sys: 406 ms, total: 3.58 s
Wall time: 5min 48s


In [6]:
df["ppp_todollar"] = df.ppp_todollar.astype(float)

In [7]:
# adjustment 
adjustment=1/(df.ppp_todollar[df.Country == 'spain'])
adjustment

57    1.333333
Name: ppp_todollar, dtype: float64

In [8]:
def ppp_to_spain(ppp_to_dollar):
    return round(ppp_to_dollar*adjustment, 2)

In [9]:
%%time
apply = df.apply(
         lambda x: ppp_to_spain(x['ppp_todollar']),
         axis=1)
df['ppp_spain']  = apply

CPU times: user 29.3 ms, sys: 1.74 ms, total: 31 ms
Wall time: 33.8 ms


In [24]:
df[df.Country == 'germany']

,Country,last_value,Currency,Code,ppp_todollar,ppp_spain
43,germany,0.744679,Euro,EUR,0.88,NaN


In [12]:
df.dtypes

Country          object
last_value      float64
Currency         object
Code             object
ppp_todollar     object
ppp_spain       float64
dtype: object

In [13]:
df["ppp_spain"] = df.ppp_spain.astype(float)

In [14]:
df[df.ppp_todollar == 0.00]

,Country,last_value,Currency,Code,ppp_todollar,ppp_spain
23,bolivia,2.694105,Mvdol,BOV,0,0.0
32,switzerland,1.143836,WIR Euro,CHE,0,0.0
33,switzerland,1.143836,Swiss Franc,CHF,0,0.0
34,switzerland,1.143836,WIR Franc,CHW,0,0.0
40,colombia,1351.775746,Unidad de Valor Real,COU,0,0.0
44,cuba,0.321939,Peso Convertible,CUC,0,0.0
56,eritrea,5.469904,Nakfa,ERN,0,0.0
66,ghana,2.032734,Ghana Cedi,GHS,0,0.0
107,mexico,9.505202,Mexican Unidad de Inversion (UDI),MXV,0,0.0
113,mongolia,898.732634,Tugrik,MNT,0,0.0


In [15]:
df.drop(df[df['ppp_todollar'] == 0.0].index, inplace = True)

# Adjustment function

In [16]:
def price_calculator(product_price, country):
    find_ppp=df.ppp_spain[df.Country == country.lower()]
    return product_price * find_ppp.item()

In [17]:
df.ppp_spain[df.Country == 'colombia'].item()

0.48

In [18]:
price_calculator(20, 'colombia')

9.6

In [19]:
conversion=convert('usd', 'eur', 60)
result = json.loads(conversion)
result['amount']

'50.26'

In [20]:
df.to_excel('/Users/mikel/Documents/Projects/chameleon-pricing/DB.xlsx', index=False)

In [28]:
df=pd.read_excel('/Users/mikel/Documents/Projects/chameleon-pricing/DB.xlsx')
df

,Country,last_value,Currency,Code,ppp_todollar,ppp_spain
0,aruba,1.350690,Aruban Florin,AWG,0.75,1.00
1,afghanistan,17.914483,Afghani,AFN,0.23,0.31
2,angola,146.167168,Kwanza,AOA,0.22,0.29
3,albania,41.342391,Lek,ALL,0.40,0.53
4,united arab emirates,2.258503,UAE Dirham,AED,0.61,0.81
...,...,...,...,...,...,...
158,uruguay,25.351299,Peso Uruguayo,UYU,0.58,0.77
159,united states,1.000000,US Dollar,USD,1.00,1.33
160,uzbekistan,2085.548458,Uzbekistan Sum,UZS,0.20,0.27
161,south africa,6.652566,Rand,ZAR,0.47,0.63


In [32]:
df[df.Country == 'brazil']

,Country,last_value,Currency,Code,ppp_todollar,ppp_spain
22,brazil,2.247384,Brazilian Real,BRL,0.45,0.6


In [36]:
import chameleonpricing as cp 
cp.price_calculator(10, 'colombia')

4.9